In [101]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# ML: Predicting Star Ratings
Objective : To predict a new venue's popularity from information available when the venue opens.  
Dataset : Venue popularities dataset provided by Yelp.  The data set contains meta data about the venue (where it is located, the type of food served, etc.).  It also contains a star rating. Note that the venues are not limited to restaurants. 

Metric: Root mean squared error of the number of stars predicted

## Download and parse the incoming data


The training data are a series of JSON objects, in a Gzipped file. 

In [102]:
import ujson as json
import gzip

with gzip.open('yelp_train_academic_dataset_business.json.gz') as f:
    data = [json.loads(line) for line in f]

In [103]:
print("Total records: ", len(data))
print("Features: \n",data[0].keys())
print("Datatype for features: ")
for d in data[0]:
    print(type(d),end=' ')

Total records:  37938
Features: 
 dict_keys(['business_id', 'full_address', 'hours', 'open', 'categories', 'city', 'review_count', 'name', 'neighborhoods', 'longitude', 'state', 'stars', 'latitude', 'attributes', 'type'])
Datatype for features: 
<class 'str'> <class 'str'> <class 'str'> <class 'str'> <class 'str'> <class 'str'> <class 'str'> <class 'str'> <class 'str'> <class 'str'> <class 'str'> <class 'str'> <class 'str'> <class 'str'> <class 'str'> 

In [104]:
# Target variable
star_ratings = [row['stars'] for row in data]

In [105]:
## Split the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data , star_ratings, test_size=0.25, random_state=0)

In [125]:
len(X_train), len(X_test)

(28453, 9485)

## Building models


Each of the "model" questions asks you to create a function that models the number of stars venues will receive.  It will be passed a list of dictionaries.  Each of these will have the same format as the JSON objects you've just read in.  Some of the keys (like the stars!) will have been removed.  This function should return a list of numbers of the same length, indicating the predicted star ratings.

### Using city_avg as the feature

The venues belong to different cities.  It can be assumed that the ratings in some cities are probably higher than others.  We wish to build an estimator to make a prediction based on this.

First, we will work out the average rating for each city and create a list of tuples (city name, star rating), one for each city in the dataset. For that, calculate the sum of the star ratings and the number of venues for each city.  At the end, we can just divide the stars by the count to get the average.

We will use collections module's `defaultdict` class that provides default values for keys that haven't been used. This will help to test whether a key exists in the dictionary before adding to the running tally.

In [106]:
from collections import defaultdict
star_sum = defaultdict(int)
count = defaultdict(int)

for row, stars in zip(data, star_ratings):
    city = row['city']
    star_sum[city]+=stars
    count[city]+=1
    
avg_stars = dict()
for city in star_sum:
    # calculate average star rating and store in avg_stars
    avg_stars[city] = star_sum[city]/count[city]

In [107]:
from itertools import islice
dict(islice(avg_stars.items(),10))
print ("Number of cities: ", len(avg_stars))

Number of cities:  167


Now, let's build a custom estimator that will make a prediction based solely on the city of a venue.  

This custom estimator will have a `.fit()` method.  It will receive `data` as its argument `X` and `star_ratings` as `y`, and should repeat the calculation of the previous problem there.  

Then the `.predict()` method can look up the average rating for the city of each record it receives.

In [108]:
from sklearn import base

class CityEstimator(base.BaseEstimator, base.RegressorMixin):
    
    def __init__(self):
        self.avg_stars = dict()
    
    def fit(self, X, y):
        # Store the average rating per city in self.avg_stars
        star_total = defaultdict(int)
        count = defaultdict(int)
        for row, star_rating in zip(X,y):
            city = row['city']
            star_total[city]+=star_rating
            count[city]+=1
            self.avg_stars[city] = star_total[city]/count[city]
        return self
    
    def predict(self, X):
        # return city average if city exists in training set, otherwise return mean of all cities
        return [self.avg_stars.get(row['city'])  if self.avg_stars.get(row['city']) else np.array([self.avg_stars[city] for city in self.avg_stars]).mean() for row in X ]


Now we can create an instance of our estimator and train it.

In [109]:
city_est = CityEstimator()
# city_est.fit(data, star_ratings)
city_est.fit(X_train, y_train)

CityEstimator()

And let's see if it works.

In [190]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, city_est.predict(X_test))

0.800321702157251

Well, the model is performing very poorly. Let's see how we can improve it

### Using business latitude and longitude as a feature

City-based model is not sufficiently fine-grained. For example, we know that some neighborhoods are trendier than others.  To solve that, let's use the latitude and longitude of a venue as features to understand neighborhood dynamics.

Instead of writing a custom estimator, we'll use one of the built-in estimators in Scikit Learn.  Since these estimators won't know what to do with a list of dictionaries, we'll build a `ColumnSelectTransformer` that will return an array containing selected keys of our feature matrix.  

We will make this transformer generic such that it works on an arbitrary list of columns making it very useful in the future as well.

In [14]:
class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col_names):
        self.col_names = col_names  # We will need these in transform()
    
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        # Return an array with the same number of rows as X and one
        # column for each in self.col_names
        X_trans = []
        for row in X:
            X_trans.append([row[col] for col in self.col_names])
        return X_trans
            

Let's test it on a single row, just as a sanity check:

In [114]:
cst = ColumnSelectTransformer(['latitude', 'longitude'])
assert (cst.fit_transform(X_train[:1])
        == [[X_train[0]['latitude'], X_train[0]['longitude']]])

Now, let's feed the output of the transformer in to a `sklearn.neighbors.KNeighborsRegressor`.  

In [115]:
from sklearn.neighbors import KNeighborsRegressor

data_transform = cst.fit_transform(X_train)
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(data_transform, y_train)


KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

As a sanity check, we'll test it with the first 5 rows.  To truly judge the performance, we'd need to make a test/train split.

In [117]:
test_data = X_train[:5]
test_data_transform = cst.transform(test_data)
knn.predict(test_data_transform), y_train[:5]

(array([4. , 3.5, 3.9, 3.7, 3.8]), [3.5, 3.0, 1.5, 3.5, 4.0])

Instead of doing this by hand, let's make a pipeline.  

In [20]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([('preprocess',cst), # ColumnSelectTransformer
                 ('model',knn)       # KNeighborsRegressor
    ])

This should work the same way.

In [118]:
pipe.fit(X_train,y_train)
pipe.predict(test_data)

array([4. , 3.5, 3.9, 3.7, 3.8])

Notice that we used `n_neighbors = 5` as hyperparameter to `KNeighborsRegressor`, which tells it how many nearest neighbors to average together when making a prediction.  
We do now know if 5 is the optimum value. So let's determine a better value of this hyperparameter.   There are several ways to do this:

1. Use [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) to split your data in to a training set and a test set.  Score the performance on the test set.  After finding the best hyperparameter, retrain the model on the full data at that hyperparameter value.

2. Use [`cross_val_score`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score) to return cross-validation scores on your data for various values of the hyperparameter.  Choose the best one, and retrain the model on the full data.

3. Use [`GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) to do the splitting, training, and grading automatically.  

<b> We will be using `GridSearchCV` </b>


In [119]:
cst = ColumnSelectTransformer(['latitude', 'longitude'])
data_transform = cst.fit_transform(X_train)

In [151]:
# Using GridSearchCV in Pipeline to compute n_neighbors value
from sklearn.model_selection import GridSearchCV

cst = ColumnSelectTransformer(['latitude', 'longitude'])
param_grid = {'n_neighbors':list(range(20,50))}
gs = GridSearchCV(KNeighborsRegressor(), param_grid, cv=7, n_jobs=2, verbose=1)
lat_long_est = Pipeline([('preprocess_data',ColumnSelectTransformer(['latitude', 'longitude'])),
                         ('grid_estimator',gs)])
lat_long_est.fit(X_train,y_train)

Fitting 7 folds for each of 30 candidates, totalling 210 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   13.8s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   55.6s
[Parallel(n_jobs=2)]: Done 210 out of 210 | elapsed:   59.8s finished


Pipeline(memory=None,
         steps=[('preprocess_data',
                 ColumnSelectTransformer(col_names=['latitude', 'longitude'])),
                ('grid_estimator',
                 GridSearchCV(cv=7, error_score='raise-deprecating',
                              estimator=KNeighborsRegressor(algorithm='auto',
                                                            leaf_size=30,
                                                            metric='minkowski',
                                                            metric_params=None,
                                                            n_jobs=None,
                                                            n_neighbors=5, p=2,
                                                            weights='uniform'),
                              iid='warn', n_jobs=2,
                              param_grid={'n_neighbors': [20, 21, 22, 23, 24,
                                                          25, 26, 27, 28, 29,
     

In [152]:
lat_long_est['grid_estimator'].best_params_

{'n_neighbors': 48}

In [191]:
yhat = lat_long_est.predict(X_test)
mean_squared_error(y_test, yhat)

0.7854394658232297

We did reduce error a little. But still there is just rather little signal available for modeling.

### Using business category as a feature:

While location is important, we could also try seeing how predictive the
venue's category is.  We'll build an estimator that considers only the categories.

The categories come as a list of strings, but the built-in estimators all need numeric input.  We will use  **one-hot encoding**, also known as dummy variables to deal with categorical features. 

The `ColumnSelectTransformer` we built earlier can be used to extract the categories column as a list of strings. 

This list of strings will be converted to a list of dictionaries using custom class `DictEncoder`

We will use Scikit Learn [`DictVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html#sklearn.feature_extraction.DictVectorizer), which takes in a list of dictionaries.  It creates a column in the output matrix for each key in the dictionary and fills it with the value associated with it.  Missing keys are filled with zeros.  Therefore, we need only build a transformer that takes a list of strings to a dictionary with keys given by those strings and values of one.

In [195]:
print(data[0]['categories'])
data[0]

['Doctors', 'Health & Medical']


{'business_id': 'vcNAWiLM4dR7D2nwwJ7nCA',
 'full_address': '4840 E Indian School Rd\nSte 101\nPhoenix, AZ 85018',
 'hours': {'Tuesday': {'close': '17:00', 'open': '08:00'},
  'Friday': {'close': '17:00', 'open': '08:00'},
  'Monday': {'close': '17:00', 'open': '08:00'},
  'Wednesday': {'close': '17:00', 'open': '08:00'},
  'Thursday': {'close': '17:00', 'open': '08:00'}},
 'open': True,
 'categories': ['Doctors', 'Health & Medical'],
 'city': 'Phoenix',
 'review_count': 7,
 'name': 'Eric Goldberg, MD',
 'neighborhoods': [],
 'longitude': -111.983758,
 'state': 'AZ',
 'stars': 3.5,
 'latitude': 33.499313,
 'attributes': {'By Appointment Only': True},
 'type': 'business'}

In [196]:
class DictEncoder(base.BaseEstimator, base.TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # X will come in as a list of lists of lists.  
        # Return a list of dictionaries corresponding to those inner lists.
        X_list=[]
        for row in X:
            categories_dict = {cat:1 for cats in row for cat in cats}
            X_list.append(categories_dict)
        return X_list
                

In [197]:
# Test DictEncoder
assert (DictEncoder().fit_transform([[['a']], [['b', 'c']]])
        == [{'a': 1}, {'b': 1, 'c': 1}])

Now we will set up a pipeline with our `ColumnSelectTransformer`, our `DictEncoder`, the `DictVectorizer`, and a regularized linear model, like `Ridge`, as the estimator.  
Also, some categories (e.g. Restaurants) are not very specific.  Others (Japanese sushi) are much more so.  One way to deal with this is using term-frequency-inverse-document-frequency (TF-IDF) between the `DictVectorizer` and the linear model.

This model will have a large number of features, one for each category, so there is a significant danger of overfitting.  We will use cross validation to choose the best regularization parameter.

In [209]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfTransformer

cst_categories = ColumnSelectTransformer(['categories'])
dict_encode = DictEncoder()
dict_vectorizer = DictVectorizer(sparse=False)
tfidf_transf = TfidfTransformer()
ridge = Ridge(alpha=0.2)

category_est = Pipeline(steps=[('col_transform', cst_categories),
                               ('dict_encode',dict_encode),
                               ('dict_vectorize', dict_vectorizer),
                               ('tfidf', tfidf_transf),
                               ('ridge',ridge)])
category_est.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('col_transform',
                 ColumnSelectTransformer(col_names=['categories'])),
                ('dict_encode', DictEncoder()),
                ('dict_vectorize',
                 DictVectorizer(dtype=<class 'numpy.float64'>, separator='=',
                                sort=True, sparse=False)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('ridge',
                 Ridge(alpha=0.2, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [210]:
y_hat = category_est.predict(X_test)

In [211]:
mean_squared_error(y_test, y_hat)

0.6590591211272805

With business categories as feature, we were able to further reduce the error.

### Model based on business attributes

There is even more information in the attributes for each venue.  Let's build an estimator based on these.

Venues attributes may be nested:
```
{
  'Attire': 'casual',
  'Accepts Credit Cards': True,
  'Ambiance': {'casual': False, 'classy': False}
}
```
We wish to encode them with one-hot encoding.  The `DictVectorizer` can do this, but only once we've flattened the dictionary to a single level, like so:
```
{
  'Attire_casual' : 1,
  'Accepts Credit Cards': 1,
  'Ambiance_casual': 0,
  'Ambiance_classy': 0
}
```

We'll build a custom transformer that flattens the attributes dictionary and place this in a pipeline with a `DictVectorizer` and a regressor.

In [56]:
class NestedDictionaryTransformer(base.BaseEstimator, base.TransformerMixin):
       
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        X_trans=[]
        for row in X:
            new_dict = dict()
            for feature in row:
                for attr,attr_val in feature.items():
                    if (isinstance(attr_val,bool)):
                        k=attr
                        v=int(attr_val)
                        new_dict[k]=v
                    elif (isinstance(attr_val,str)):
                        k = attr+'_'+attr_val
                        v = 1
                        new_dict[k]=v
                    elif (isinstance(attr_val,dict)):
                        for key,val in attr_val.items():
                            k = attr+"_"+key
                            if (isinstance(val,bool)):
                                v=int(val)
                            elif (isinstance(attr_val,str)):
                                v = 1
                            new_dict[k]=v
#                 row[j]=new_dict
            X_trans.append(new_dict)
        return X_trans

Lets test this

In [174]:
ndt = NestedDictionaryTransformer()
ndt_t = DictVectorizer(sparse=False).fit_transform( \
                                                   ndt.fit_transform( \
                                                                     ColumnSelectTransformer(['attributes']).transform(X_train)))


In [202]:
ndt_t[3]

array([1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0.])

It will be difficult to find a single regressor that does well enough.  We can address this by using a linear model to fit the linear part of data, and use a non-linear model to fit the residual that the linear model can't fit.  That is, build a residual estimator that takes as an argument two other estimators.  It should use the first to fit the raw data and the second to fit the residuals of the first.

In [203]:
import seaborn as sns
from sklearn import tree
from sklearn import metrics

class LinearAndNonlinearEstimator(base.BaseEstimator, base.RegressorMixin):
    
    def __init__(self,linear, non_linear):
        self.avg_stars = []
        self.non_linear = non_linear
        self.linear = linear
    
    def fit(self, X, y):
        self.linear.fit(X,y)
        y_hat = self.linear.predict(X)
        residual = y-y_hat
        self.non_linear.fit(X, residual)
        
    def predict(self, X):
        return self.linear.predict(X)+self.non_linear.predict(X)

#### Transform data

In [207]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.model_selection import GridSearchCV

cst_attribute = ColumnSelectTransformer(['attributes'])
nested_dic_transformer = NestedDictionaryTransformer()
dict_vectorizer = DictVectorizer(sparse=False)

pipe_transformed = Pipeline(steps=[('cst_attributes', cst_attribute),
                                  ('nested_dic_transformer',nested_dic_transformer),
                                  ('dict_vectorizer',dict_vectorizer)])
X_t = pipe_transformed.fit_transform(X_train,y_train)

#### Use GridSearchCV to compute best parameters for linear regressor

In [222]:
# forest = RandomForestRegressor(n_estimators=80, min_samples_split=15,random_state=100)
gs_ridge = GridSearchCV(Ridge(),param_grid={'alpha':list(range(1,40))}, cv=5, verbose=1, n_jobs=2)

gs_ridge.fit(X_t,y_train)

Fitting 5 folds for each of 39 candidates, totalling 195 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done 123 tasks      | elapsed:    4.2s
[Parallel(n_jobs=2)]: Done 195 out of 195 | elapsed:    4.6s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=2,
             param_grid={'alpha': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                   14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
                                   25, 26, 27, 28, 29, 30, ...]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [179]:
gs_ridge.best_params_

{'alpha': 15}

In [224]:
y_ridge = gs_ridge.predict(X_t)

In [226]:
y_residual = y_train - y_ridge

#### Use GridSearchCV to compute best parameters for non-linear regressor

In [227]:
forest_params = {'n_estimators':[10,50,75,100],
                 'min_samples_split':list(range(5,60,5))}
gs_forest = GridSearchCV(RandomForestRegressor(),param_grid=forest_params, cv=5, verbose=1, n_jobs=2)
gs_forest.fit(X_t,y_residual)
gs_forest.best_params_

Fitting 5 folds for each of 44 candidates, totalling 220 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  1.0min
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:  4.1min
[Parallel(n_jobs=2)]: Done 220 out of 220 | elapsed:  4.6min finished


{'min_samples_split': 55, 'n_estimators': 100}

Now that we have the hyperparameters from GridSearch, we will build a pipeline for data transformation and model.

In [228]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

cst_attribute = ColumnSelectTransformer(['attributes'])
nested_dic_transformer = NestedDictionaryTransformer()
dict_vectorizer = DictVectorizer(sparse=False)

forest = RandomForestRegressor(n_estimators=100, min_samples_split=55,random_state=0)
lnl = LinearAndNonlinearEstimator(Ridge(alpha=15),forest)
pipe_attributes = Pipeline(steps=[('cst_attributes', cst_attribute),
                                  ('nested_dic_transformer',nested_dic_transformer),
                                  ('dict_vectorizer',dict_vectorizer),
                                  ('tfidf', TfidfTransformer()),
                                  ('lnl',lnl)])

pipe_attributes.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('cst_attributes',
                 ColumnSelectTransformer(col_names=['attributes'])),
                ('nested_dic_transformer', NestedDictionaryTransformer()),
                ('dict_vectorizer',
                 DictVectorizer(dtype=<class 'numpy.float64'>, separator='=',
                                sort=True, sparse=False)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('lnl',
                 L...
                                                          tol=0.001),
                                             non_linear=RandomForestRegressor(bootstrap=True,
                                                                              criterion='mse',
                                                                              max_depth=None,
                                                                     

In [229]:
yhat = pipe_attributes.predict(X_test)
mean_squared_error(y_test, yhat)

0.7340276030103203

## Create a full model using all the features

So far we have only built models based on individual features.  Now we will build an ensemble regressor that averages together the estimates of the four previous regressors.

In order to use the existing models as input to an estimator, we will have to turn them into transformers.  (A pipeline can contain at most a single estimator.)  
- We will build a custom `ModelTransformer` class that takes an estimator as an argument.  When `fit()` is called, the estimator should be fit.  When `transform()` is called, the estimator's `predict()` method should be called, and its results returned.


In [68]:
class EstimatorTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, estimator):
        self.estimator=estimator
    
    def fit(self, X, y):
        # Fit the stored estimator.
        self.estimator.fit(X,y)
        return self # for chaining, its important to return self
    
    def transform(self, X):
        # Use predict on the stored estimator as a "transformation".
        return np.array(self.estimator.predict(X)).reshape(-1,1) # transform() should be a 2-D array with a single column

This should work as follows:

In [215]:
city_trans = EstimatorTransformer(city_est)
city_trans.fit(X_train, y_train)

assert ([r[0] for r in city_trans.transform(X_train[:5])]
        == city_est.predict(X_train[:5]))

Create an instance of `ModelTransformer` for each of the previous four models. Combine these together in a single feature matrix with a
[`FeatureUnion`](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html#sklearn.pipeline.FeatureUnion).

In [185]:
from sklearn.pipeline import FeatureUnion
city_est = CityEstimator()

lat_long_est = Pipeline([('preprocess_data',ColumnSelectTransformer(['latitude', 'longitude'])),
                         ('optimized_model',KNeighborsRegressor(n_neighbors=30))])

category_est = Pipeline(steps=[('col_transform',  ColumnSelectTransformer(['categories'])),
                               ('dict_encode',DictEncoder()),
                               ('dict_vectorize', DictVectorizer(sparse=False)),
                               ('tfidf', TfidfTransformer()),
                               ('ridge',Ridge(alpha=0.2))])


pipe_attributes = Pipeline(steps=[('cst_attributes', ColumnSelectTransformer(['attributes'])),
                                  ('nested_dic_transformer',NestedDictionaryTransformer()),
                                  ('dict_vectorizer',DictVectorizer(sparse=False)),
                                  ('lnl',LinearAndNonlinearEstimator(Ridge(alpha=16),RandomForestRegressor(n_estimators=100, min_samples_split=35,random_state=100)))])
union = FeatureUnion([
        # FeatureUnions use the same syntax as Pipelines
    ('city_est',EstimatorTransformer(city_est)),
    ('lat_long_est',EstimatorTransformer(lat_long_est)),
    ('category_est', EstimatorTransformer(category_est)),
    ('pipe_attributes', EstimatorTransformer(pipe_attributes))
    ])

This should return a feature matrix with four columns.

In [186]:
union.fit(X_train,y_train)
trans_data = union.transform(X_train[:10])
assert trans_data.shape == (10, 4)
print ("trans_data: ",trans_data)

trans_data:  [[3.64940264 3.95       3.79522399 3.95081885]
 [3.64940264 3.61666667 3.44200052 3.44874792]
 [3.64940264 3.93333333 3.20695889 3.95081885]
 [3.65549005 3.43333333 3.27679335 3.38878487]
 [3.66666667 3.76666667 3.21105972 4.10628361]
 [3.67213598 4.25       3.96991437 3.67892706]
 [3.65549005 3.43333333 3.41878343 3.55544491]
 [3.70606061 4.03333333 4.27592511 3.95081885]
 [3.816875   3.55       4.06436874 3.67892706]
 [3.67213598 3.81666667 3.79925182 3.77252277]]


Finally, use a pipeline to combine the feature union with a linear regression model to weight the predictions.

By combining our models with a linear model, we will be unable to notice any correlation between features.  We don't expect all attributes to have the same effect on all venues.  For example, "Ambiance: divey" might be a bad indicator for a restaurant but a good one for a bar.  Nonlinear models can pick up on this interaction.  We'll use a linear model and then a nonlinear model to fit the residuals of the linear model.

In [230]:
X_t = union.transform(X_train)
lr = LinearRegression()
yhat_lr = lr.fit(X_t,y_train).predict(X_t)
yhat_res = y_train - yhat_lr

In [232]:
grid_lin = GridSearchCV(RandomForestRegressor(), param_grid={'n_estimators': [10,50,80,100,120], 'min_samples_split':list(range(5,60,5))})
grid_lin.fit(X_t, yhat_res)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'min_samples_split': [5, 1

In [233]:
grid_lin.best_params_

{'min_samples_split': 55, 'n_estimators': 120}

In [219]:
full_est = Pipeline([('union',  union),
                     ('lin',LinearAndNonlinearEstimator(LinearRegression(),RandomForestRegressor(n_estimators=100, min_samples_split=35,random_state=100)))])
full_est.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('union',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('city_est',
                                                 EstimatorTransformer(estimator=CityEstimator())),
                                                ('lat_long_est',
                                                 EstimatorTransformer(estimator=Pipeline(memory=None,
                                                                                         steps=[('preprocess_data',
                                                                                                 ColumnSelectTransformer(col_names=['latitude',
                                                                                                                                    'longitude'])),
                                                                                                ('optimized_model',
                                                          

In [220]:
y_hat = full_est.predict(X_test)
mean_squared_error(y_test, y_hat) #0.644

0.6568391283888111